# Data Cleaning project

In [1]:
# Cleaning will be performed mainly with Pandas.
import pandas as pd

## Data set
The data set is an extract from American `itdashboard.gov` archive. It containt information about governmental IT projects, their dates, costs, stakeholders (i.e. specific departments).

In [2]:
df = pd.read_csv('dataset.csv', index_col=False)

## Cleaning columns one by one
1. `Unique Investment Identifier`:
    1. Not unique, despite the name. It shouldn't be the first column, because it looks like Primary Key.
	2. **Identifier** could be **ID**.

In [3]:
# Rename the column
df.rename(columns={'Unique Investment Identifier':'Investment ID'}, inplace=True)

2. `Business Case ID`
    1. It is float, but should be int.
    2. It corresponds 1:1 with `Investment ID`, so it should be kept in another lookup table. The same with `Investment Title`.
    3. There are some `NaN` values. These are rows with totals and they should be removed - they don't add any value, but only complicate an analysis.

In [4]:
# Drop rows with totals
indices_to_drop = df.loc[df['Business Case ID'].isna()].index
df.drop(indices_to_drop, inplace=True)

# Change type to int
df['Business Case ID'] = df['Business Case ID'].astype(int)

# Save into another DF
investments = pd.DataFrame(data=df[['Investment ID', 'Business Case ID', 'Investment Title']])
investments.drop_duplicates(inplace=True)
investments.set_index(keys='Investment ID', inplace=True)

# Export into a file
investments.to_csv('investments.csv')

# Drop extracted columns
df.drop(labels=['Business Case ID', 'Investment Title'], axis=1, inplace=True)

3. `Agency Code`
    1. **Code** could be **ID**.
	2. It is float, but should be int.

In [5]:
# Rename the column to `Agency ID`:
df.rename(columns={'Agency Code':'Agency ID'}, inplace=True)

# Change type to int
df['Agency ID'] = df['Agency ID'].astype(int)

4. `Agency Name`
	1. It corresponds 1:1 with `Agency Code`, so it should be kept in another lookup table. In fact there are only two cases in whole DataFrame, where `Agency ID` doesn't reflect `Agency Name` ideally. By comparing lengths of `Agency ID`, `Agency Name` and pair of these two, we see that there is **one more of unique entries** within Code+Name pair. By checking uniqueness we can conclude that value `6` is our point of pain. Two entries, which have wrong `Agency ID` values (not `Agency Name`, what can be easily deduced from other entries), should have `5`. 
    2. Names should be extracted to separate table.

In [6]:
# Define indices of rows to be changed
indices_to_change = df[(df['Agency ID'] == 6) & (df['Agency Name'] != 'Department of Commerce')].index

# Assign new values
for index in indices_to_change:
    df.at[index, 'Agency ID'] = 5
    
# Save into another DF
col_agencies = ['Agency ID', 'Agency Name']
agencies = pd.DataFrame(data=df[col_agencies])
agencies.drop_duplicates(inplace=True)

# Export into a file
agencies.to_csv('agencies.csv')

# Drop extracted columns
df.drop(labels=['Agency Name'], axis=1, inplace=True)

5. `Investment Title`
    1. It has been extracted with `Investment ID`.

6. `Project ID`
    1. It's a float, but should be an int.
	2. It's unique, so it should be the Primary Key and the first column in data set. This fits also the business context of the whole data set. 

In [7]:
# Change type to int
df['Project ID'] = df['Project ID'].astype(int)

# Set index
df.set_index(keys='Project ID', inplace=True)

7. `Agency Project ID`
	1. Almost half of the values are NaN (but read by Pandas correctly). Since these are some IDs of internal use of agencies, they won't be touched.
    2. Should be extrated with to separate table.

In [8]:
# Save into another DF
col_agency_projects = ['Agency ID', 'Agency Project ID']
agency_projects = pd.DataFrame(data=df[col_agency_projects].dropna())
agency_projects.set_index(keys='Agency Project ID', inplace=True)
agency_projects.drop_duplicates(inplace=True)

# Export into a file
agency_projects.to_csv('agency_projects.csv')

# Drop extracted columns
df.drop(labels=['Agency ID'], axis=1, inplace=True)

8. `Project Name`
	1. It corresponds with `Project ID`, so it should be kept in another lookup table with `Project Description` (see next point) in a `project_descriptions` table.

9. `Project Description`
	1. It corresponds with `Project ID`, so it should be kept in another lookup table with `Project Name` in a `project_descriptions` table.

In [9]:
# Save into another DF
project_descriptions = pd.DataFrame(data=df[['Project Name', 'Project Description']])
project_descriptions.drop_duplicates(inplace=True)

# Export into a file
project_descriptions.to_csv('project_descriptions.csv')

# Drop extracted columns
df.drop(labels=['Project Name', 'Project Description'], axis=1, inplace=True)

### Date columns
10. `Start Date`, 11. `Completion Date (B1)`, 12. `Planned Project Completion Date (B2)`, 13. `Projected/Actual Project Completion Date (B2)`
	1. They are objects (Pandas' string), but should be dates.
    2. All dates should be extracted to separate table (not obligatory, but it's good not to mix a business contexts).
    3. Multiple formats (`2012-30-09`, `31/03/2012`)
    4. Column names too long.
    5. Since there also `Completion Date (B1)`, `Planned Project Completion Date (B2)` and `Projected/Actual Project Completion Date (B2)`, a purpose of these columns is unobvious. However the full business context is not known, so they won't be deleted or merged.

In [10]:
import re

# Rename the columns
col_renames = {'Start Date':'Start',
               'Completion Date (B1)':'Completion',
               'Planned Project Completion Date (B2)':'Planned Completion',
               'Projected/Actual Project Completion Date (B2)':'Projected/Actual Completion'}
df.rename(columns=col_renames, inplace=True)

# Define a parser for two different formats
pattern = re.compile('[0-9]{4}-[0-9]{2}-[0-9]{2}')

def date_parser(arg):
    global pattern
    if pattern.match(str(arg)):
        return pd.to_datetime(arg=arg, format='%Y-%d-%m')
    return pd.to_datetime(arg=arg)

# Parse dates
col_dates = ['Start', 'Completion', 'Planned Completion', 'Projected/Actual Completion']

for col in col_dates:
    df[col] = df[col].apply(date_parser)

# Extract dates
dates = pd.DataFrame(data=df[col_dates])

# Export into a file
dates.to_csv('dates.csv')

# Drop extracted columns
df.drop(labels=col_dates, axis=1, inplace=True)

### Cost columns
14. `Planned Cost ($ M)`, 15. `Projected/Actual Cost ($ M)`
	1. Columns should be extracted to separate table (not obligatory, but it's good not to mix a business contexts).

In [11]:
col_costs = ['Planned Cost ($ M)', 'Projected/Actual Cost ($ M)']

# Extract costs
costs = pd.DataFrame(data=df[col_costs])

# Export into a file
costs.to_csv('costs.csv')

# Drop extracted columns
df.drop(labels=col_costs, axis=1, inplace=True)

### Update columns
16. `Updated Date`, 17. `Updated Time`
	1. Should be merged together as a timestamp.

In [12]:
df['Updated'] = df['Updated Date'] + " " + df['Updated Time']
df['Updated'] = pd.to_datetime(arg=df['Updated'], format='%d/%m/%Y %H:%M:%S')

# Drop merged columns
df.drop(labels=['Updated Date', 'Updated Time'], axis=1, inplace=True)

## Output cleaned tables
1. 

In [13]:
df.sort_index(inplace=True)